MU MIMOシミュレーション
===

In [3]:
# import
import numpy as np
import sys
import matplotlib.pyplot as plt

np.set_printoptions(precision=4, suppress=True, floatmode="maxprec_equal")
plt.rcParams['figure.figsize'] = (12,4)

In [4]:
N_T = 4
N_R = 2
Ndata = 1000
SNRdB = 100

In [5]:
def rndCode(codeSize, Type):
    """
    codeSize: 生成するランダム系列の大きさ（行数ｘ列数）を指定
    Type: タイプ指定 0: [0,1], 1: [-1, 1]
    """
    
    rndCode = np.random.randn(*codeSize)
    
    if rndCode.ndim > 2:
        sys.exit()
    
    dataSize = rndCode.shape
    if len(dataSize) == 1:
        sys.exit()
    
    if Type == 0:
        rndCode = np.where(rndCode <= 0, 0, 1)
    else:
        rndCode = np.where(rndCode <= 0, -1, 1)
    
    return rndCode

# BPSK変調器
def bpskMod(data):
    """
    data: 列ベクトル
    bpskSymbol： 列ベクトル
    """
    bpskSymbol = data.copy()
    np.place(bpskSymbol, bpskSymbol == 0, -1)
    
    return bpskSymbol

# ガウス雑音生成

def awgn(Pn, rn, cn):
    """
    Pn: 雑音電力
    ｒｎ: 行数
    cn: 列数
    """
    n = np.random.randn(rn, cn) + 1j*np.random.randn(rn, cn)
    n = n * np.sqrt(Pn/2)
    
    return n

# 遅延発生器
def delayGen(delayVec, ssSigMat):
    sigMat = ssSigMat.copy()
    Nsig = delayVec.shape[1]
    
    if Nsig != sigMat.shape[1]:
        sys.exit(1)
    
    for co in range(0, Nsig):
        if delayVec[0,co] > 0:
            roll_num = delayVec[0][co]
            sigMat[:,co] = np.roll(sigMat[:,co],roll_num)
            sigMat[:roll_num, co] = 0
            
    return sigMat

# BPSK復調器
def bpskDem(rSig):
    """
    rSig : 受信信号
    rData:　受信データ
    """
    rData = np.ones(rSig.shape)
    rData[rSig < 0] =  0
    
    return rData

# BER比較器
def ber(data1, data2):
    BER = np.sum(np.abs(data1-data2))/max(data1.shape)
    return BER

In [7]:
# 送信機
data_A = rndCode([N_R, Ndata], 0)
data_B = rndCode([N_R, Ndata], 0)
bpskSymbols_A = bpskMod(data_A);
bpskSymbols_B = bpskMod(data_B);

bpskSymbols = np.vstack([bpskSymbols_A, bpskSymbols_B])

In [8]:
#通信経路 
H_A = awgn(1, N_R, N_T);
H_B = awgn(1, N_R, N_T);

In [27]:
#　MU　MIMO　重み係数を計算するための下準備
[V_R_A, Lambda_A, V_H_A] = np.linalg.svd(H_A);
V_T_A = V_H_A.conj().T
Vn_A = V_T_A[:, 2:4]
[V_R_B, Lambda_B, V_H_B] = np.linalg.svd(H_B);
V_T_B = V_H_B.conj().T
Vn_B = V_T_B[:, 2:4]
H_A_tilda = H_A @ Vn_B
H_B_tilda = H_B @ Vn_A
[V_R_A_tilda, Lambda_A_tilda, V_H_A_tilda] = np.linalg.svd(H_A_tilda)
[V_R_B_tilda, Lambda_B_tilda, V_H_B_tilda] = np.linalg.svd(H_B_tilda)
V_T_A_tilda = V_H_A_tilda.conj().T
V_T_B_tilda = V_H_B_tilda.conj().T

In [28]:
#　送信機
W_T_A = Vn_B @ V_T_A_tilda
W_T_B = Vn_A @ V_T_B_tilda
U_A = W_T_A @ bpskSymbols_A
U_B = W_T_B @ bpskSymbols_B

In [31]:
#通信路と受信信号
Pn = 10 ** (-SNRdB/10) * Lambda_A_tilda[0];
X_A = H_A @ (U_A + U_B) + awgn(Pn, N_R, Ndata);
X_B = H_B @ (U_A + U_B) + awgn(Pn, N_R, Ndata);

In [32]:
#　受信機
Y_A = V_R_A_tilda.conj().T @ X_A;
Y_B = V_R_B_tilda.conj().T @ X_B;
rData_A = bpskDem(Y_A)
rData_B = bpskDem(Y_B)

In [34]:
#　BER計算と結果表示
for pipeCo in range(0, N_R):
    BER_A = ber(data_A[pipeCo, :], rData_A[pipeCo,:]);
    BER_B = ber(data_B[pipeCo, :], rData_B[pipeCo,:]);
    print(BER_A, BER_B)

0.0 0.0
0.0 0.0
